In [1]:
import re
from pycoingecko import CoinGeckoAPI
from datetime import datetime
import pandas as pd

In [2]:
cg = CoinGeckoAPI()

In [3]:
def utc_to_timestamp(data):
    '''
    Get the date from the format dd/mm/yyyy and convert to timestamp
    '''
    #Get day, month, year
    data = data.split('/')
    day = re.sub('^0','',data[0])
    month = re.sub('^0','',data[1])
    year = data[2]

    #Put into the yyyy/mm/dd format
    date = datetime(int(year), int(month), int(day))

    #Remove decimal part
    
    return '{:.0f}'.format(datetime.timestamp(date))

def timestamp_to_utc(data):
    '''
    Get the date from the format timestamp, Ex: 1619913921870, and convert to format dd/mm/yyy, in a tuple
    '''
    data = datetime.fromtimestamp(int(data)/1000)

    #Get day, month, year
    day = '{:02.0f}'.format(data.day)
    month = '{:02.0f}'.format(data.month)
    year = '{:04.0f}'.format(data.year)
    hour = '{:02.0f}'.format(data.hour)
    minute = '{:02.0f}'.format(data.minute)

    return (day,month,year,hour,minute)

def adjust_date(data):
    new_dict = {}
    for categories, values in data.items():
        new_list = []
        for stamp in values:
            day,month,year,hour,minute=timestamp_to_utc(stamp[0])
            new_list.append([f'{day}/{month}/{year} {hour}:{minute}', stamp[1]])
        new_dict[categories] = new_list
    return new_dict

In [4]:
data_ini = '02/05/2021'
data_fim = '03/05/2021'

stamp_ini = utc_to_timestamp(data_ini)
stamp_fim = utc_to_timestamp(data_fim)

ethereum_data = cg.get_coin_market_chart_range_by_id(id='ethereum', vs_currency='usd', from_timestamp=stamp_ini, to_timestamp=stamp_fim)
ethereum_data_utc = adjust_date(ethereum_data)
df = pd.DataFrame(ethereum_data_utc['prices'], columns=['Datetime','Price'])
df.describe()

,Price
count,24.000000
mean,2940.194259
std,32.771986
min,2880.925695
25%,2919.414460
50%,2931.037424
75%,2958.345681
max,3020.633947
